In [422]:
import csv
import pandas as pd
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.graphics.factorplots import interaction_plot
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
%matplotlib inline  

In [423]:
barcode2hpv={}
for line in open('../data/hpv_combo.txt').read().rstrip().split('\n'):
    row=line.split('\t')
    if row[1]=='True':
        barcode2hpv[row[0]]='1'
    elif row[1]=='False':
        barcode2hpv[row[0]]='0'

In [424]:
id2barcode={}
for line in open('../data/HNSCC_Chicago/tumor_normal_IDs.csv').read().rstrip().split('\n'):
    row=line.split('\t')
    id2barcode[row[0]]=row[1].split('"')[1]

In [425]:
col={}
with open('../data/HNSCC_Chicago/clin_no_name.txt', 'rb') as f:
    reader = csv.reader(f)
    for row in reader:
        if row[0]=='':
            for i in range(len(row)):
                col[row[i]]=i
            continue
        Sample_ID=row[col['Sample.ID']]
        HPV=row[col['HPV']]
        if Sample_ID in id2barcode and HPV in ['pos','neg']:
            if HPV=='pos':
                barcode2hpv[id2barcode[Sample_ID]]='1'
            elif HPV=='neg':
                barcode2hpv[id2barcode[Sample_ID]]='0'

In [426]:
for line in open('../data/hpv_CESC.txt').read().rstrip().split('\n'):
    row=line.split('\t')
    if row[1]=='negative':
        barcode2hpv[row[0]]='0'
    else:
        barcode2hpv[row[0]]='1'

In [427]:
DIR4NBS="/cellar/data/users/wzhang1984/forNBS/"
oncogene_tsg={}
for line in open(DIR4NBS+"oncogene_tsg.txt").read().rstrip().split("\n"):
    a=line.split("\t")
    oncogene_tsg[a[0]]=a[1]

In [428]:
coding_genes=set()
for line in open("/cellar/data/users/wzhang1984/bcbio/genomes/Hsapiens/GRCh37/rnaseq-2014-07-14/ref-transcripts.gtf"):
    a=line.split("\t")
    if a[1]!="protein_coding" or a[2]!="transcript":
        continue
    gene_name=a[-1].split('gene_name "')[1].split('"')[0]
    if gene_name:
        coding_genes.add(gene_name)

# Parse MAF

In [429]:
def parse_maf(fn, cohort):
    
    df = pd.read_table(fn, low_memory=False)
    
    df = df.loc[(df.loc[:,'is_flank']==0) & (df.loc[:,'is_silent']==0),:]
    if cohort[:4] == 'TCGA':
        df['pat'] = df.loc[:,'Tumor_Sample_Barcode'].str[:12]
    elif cohort == 'Chicago':
        df['pat'] = df.loc[:,'Tumor_Sample_Barcode'].str[:-2]
        
    filter_rows = []
    genes = set()
    for index, row in df.iterrows():
        gene = row['Hugo_Symbol']
        genes.add(gene)
        VC = row['Variant_Classification']
        if gene in oncogene_tsg:
            if oncogene_tsg[gene] in ['Oncogene']:
                if VC not in ['Missense_Mutation', 'In_Frame_Del', 'In_Frame_Ins', 'De_novo_Start_InFrame']:
                    filter_rows.append(index)
            if oncogene_tsg[gene]=='Amplification_Oncogene':
                filter_rows.append(index)
    df = df.drop(filter_rows)
    
    df = df.loc[:,['pat','Hugo_Symbol']]
    
    df['counter']=1
    df.set_index(['pat','Hugo_Symbol'],inplace=True)
    
    df=df.counter.groupby(level=[0, 1]).min().unstack()
    df.fillna(0,inplace=True)
    
    return df, genes

In [430]:
fn='/cellar/data/users/wzhang1984/Firehose/Firehose__2015_08_21/analyses/HNSC/Mutation_Assessor/HNSC-TP.maf.annotated'
df, genes1 = parse_maf(fn, 'TCGA-HNSC')

In [431]:
fn='../data/HNSCC_Chicago/chicago_merge.final_analysis_set.maf'
df2, genes2 = parse_maf(fn, 'Chicago')

In [433]:
fn='/cellar/data/users/wzhang1984/Firehose/Firehose__2015_08_21/analyses/CESC/Mutation_Assessor/CESC-TP.maf.annotated'
df3, genes3 = parse_maf(fn, 'TCGA-CESC')

In [434]:
coding_genes = coding_genes | genes1| genes2 | genes3

# Parse CNA

In [435]:
def parse_CNA(fn, cohort, genes):
    df = pd.read_table(fn,low_memory=False,index_col=0)
    
    df = df[df.index.map(lambda x: x in genes)]
    
    if cohort[:4] == 'TCGA':
        df = df.iloc[:,2:]
        df = (df/2.).round(0)
        df.columns = df.columns.str[:12]
    elif cohort == 'Chicago':
        df.columns = [id2barcode[i] for i in df.columns]
    
    nonOncogene_rows = []
    for index, row in df.iterrows():
        gene = index
        if not (gene in oncogene_tsg and oncogene_tsg[gene] in ['Oncogene', 'Amplification_Oncogene']):
            nonOncogene_rows.append(index)
    df.loc[nonOncogene_rows,:] = df.loc[nonOncogene_rows,:]*(-1)
    df = df.clip(lower=0)
    return df

In [436]:
fn = '/cellar/data/users/wzhang1984/Firehose/Firehose__2015_08_21/analyses/HNSC/CopyNumber_Gistic2/all_thresholded.by_genes.txt'
df4 = parse_CNA(fn, 'TCGA-HNSC', coding_genes)

In [437]:
fn = '../data/HNSCC_Chicago/CNA.table.20131119.txt'
df5 = parse_CNA(fn, 'Chicago', coding_genes)

In [438]:
fn = '/cellar/data/users/wzhang1984/Firehose/Firehose__2015_08_21/analyses/CESC/CopyNumber_Gistic2/all_thresholded.by_genes.txt'
df6 = parse_CNA(fn, 'TCGA-CESC', coding_genes)

# Concatenate results

In [439]:
df = pd.concat([df,df4.transpose()], axis=1,
               join='inner').transpose().groupby(level=0).sum().clip(upper=1.).transpose()
df = df.loc[set(df.index) & set(df4.columns) & set(barcode2hpv.keys()),df4.index]

In [440]:
df2 = pd.concat([df2,df5.transpose()], axis=1,
                join='inner').transpose().groupby(level=0).sum().clip(upper=1.).transpose()
df2 = df2.loc[set(df2.index) & set(df5.columns) & set(barcode2hpv.keys()),df5.index]

In [441]:
df3 = pd.concat([df3,df6.transpose()], axis=1,
                join='inner').transpose().groupby(level=0).sum().clip(upper=1.).transpose()
df3 = df3.loc[set(df3.index) & set(df6.columns) & set(barcode2hpv.keys()),df6.index]

In [442]:
df = pd.concat([df,df2,df3])

In [443]:
disease=[]
HPV=[]
Chicago=[]
for index, row in df.iterrows():
    if barcode2hpv[index]=='0':
        HPV.append(0)
    else:
        HPV.append(1)
    if index in df3.index:
        disease.append(0)
    else:
        disease.append(1)
    if index in df2.index:
        Chicago.append(1)
    else:
        Chicago.append(0)

df['disease'] = disease
df['Chicago'] = Chicago
df['HPV'] = HPV

In [444]:
df.loc[:,['disease','Chicago','HPV']].to_csv('../data/network_fig3/forOncoprint/sample_order.txt',sep='\t')

# Logistic regression - Anova Chisq - R solution

In [380]:
# Write the input file for logistic regression in R code
df.to_csv('../data/glmBinomial_anovaChisq/df4glm_HNSC_CESC.txt',sep='\t')

# For oncoprinter visualization

In [410]:
def parse_maf_oncoprinter(fn, cohort):
    
    df = pd.read_table(fn, low_memory=False)
    
    df = df.loc[(df.loc[:,'is_flank']==0) & (df.loc[:,'is_silent']==0),:]
    if cohort[:4] == 'TCGA':
        df['Sample'] = df.loc[:,'Tumor_Sample_Barcode'].str[:12]
    elif cohort == 'Chicago':
        df['Sample'] = df.loc[:,'Tumor_Sample_Barcode'].str[:-2]
        
    filter_rows = []
    df['Type']=''
    for index, row in df.iterrows():
        gene = row['Hugo_Symbol']
        VC = row['Variant_Classification']
        if gene in oncogene_tsg and oncogene_tsg[gene]=='Amplification_Oncogene':
            filter_rows.append(index)
        if VC not in ['Missense_Mutation', 'In_Frame_Del', 'In_Frame_Ins', 'De_novo_Start_InFrame']:
            if gene in oncogene_tsg and oncogene_tsg[gene]=='Oncogene':
                filter_rows.append(index)
            df.loc[index,'Type']='TRUNC'
        else:
            df.loc[index,'Type']='MISSENSE'
    df = df.drop(filter_rows)
    df['Cohort']=cohort
    
    df = df.loc[:,['Sample','Hugo_Symbol','Protein_Change','Type','Cohort']]
    
    return df

In [411]:
fn='/cellar/data/users/wzhang1984/Firehose/Firehose__2015_08_21/analyses/HNSC/Mutation_Assessor/HNSC-TP.maf.annotated'
df_oncoprinter = parse_maf_oncoprinter(fn, 'TCGA-HNSC')

In [412]:
fn='../data/HNSCC_Chicago/chicago_merge.final_analysis_set.maf'
df2_oncoprinter = parse_maf_oncoprinter(fn, 'Chicago')

In [413]:
fn='/cellar/data/users/wzhang1984/Firehose/Firehose__2015_08_21/analyses/CESC/Mutation_Assessor/CESC-TP.maf.annotated'
df3_oncoprinter = parse_maf_oncoprinter(fn, 'TCGA-CESC')

In [414]:
df_oncoprinter.to_csv('../data/network_fig3/forOncoprinter.txt',sep='\t',index=False,mode='w')
df2_oncoprinter.to_csv('../data/network_fig3/forOncoprinter.txt',sep='\t',index=False,mode='a',header=False)
df3_oncoprinter.to_csv('../data/network_fig3/forOncoprinter.txt',sep='\t',index=False,mode='a',header=False)

In [415]:
def parse_CNA_oncoprinter(fn, cohort):
    df = pd.read_table(fn,low_memory=False,index_col=0)
    if cohort[:4] == 'TCGA':
        df = df.iloc[:,2:]
        df = (df/2.).round(0)
        df.columns = df.columns.str[:12]
    elif cohort == 'Chicago':
        df.columns = [id2barcode[i] for i in df.columns]
    nonOncogene_rows = []
    line_out='Sample\tGene\tAlteration\tType\tCohort\n'
    for index, row in df.iterrows():
        gene = index
        if not (gene in oncogene_tsg and oncogene_tsg[gene] in ['Oncogene', 'Amplification_Oncogene']):
            nonOncogene_rows.append(index)
    df.loc[nonOncogene_rows,:] = df.loc[nonOncogene_rows,:]*(-1)
    df = df.clip(lower=0)
    df.loc[nonOncogene_rows,:] = df.loc[nonOncogene_rows,:]*(-1)
    df=pd.DataFrame(df.transpose().replace(0,np.nan).stack())
    df.replace(1,'AMP',inplace=True)
    df.replace(-1,'HOMDEL',inplace=True)
    df.columns=['Alteration']
    df['Type']='CNA'
    df['Cohort']=cohort
    return df  


In [416]:
fn = '/cellar/data/users/wzhang1984/Firehose/Firehose__2015_08_21/analyses/HNSC/CopyNumber_Gistic2/all_thresholded.by_genes.txt'
df4_oncoprinter = parse_CNA_oncoprinter(fn, 'TCGA-HNSC')

In [418]:
fn = '../data/HNSCC_Chicago/CNA.table.20131119.txt'
df5_oncoprinter = parse_CNA_oncoprinter(fn, 'Chicago')

In [419]:
fn = '/cellar/data/users/wzhang1984/Firehose/Firehose__2015_08_21/analyses/CESC/CopyNumber_Gistic2/all_thresholded.by_genes.txt'
df6_oncoprinter = parse_CNA_oncoprinter(fn, 'TCGA-CESC')

In [420]:
df4_oncoprinter.to_csv('../data/network_fig3/forOncoprinter.txt',sep='\t',mode='a',header=False)
df5_oncoprinter.to_csv('../data/network_fig3/forOncoprinter.txt',sep='\t',mode='a',header=False)
df6_oncoprinter.to_csv('../data/network_fig3/forOncoprinter.txt',sep='\t',mode='a',header=False)